In [54]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import pickle
from exp3 import EXP3
from scipy.stats import ttest_rel

## Get data

In [55]:
with open('sp100tickers.pickle','rb') as f:
    tickers = pickle.load(f)
data = yf.download(list(tickers), start='2017-01-01', end='2022-01-01')
data = pd.DataFrame(data)
data = data['Close'].reset_index(drop = True)
data = data.pct_change().dropna().reset_index(drop = True)

[*********************100%***********************]  100 of 100 completed


## Experiment with SP100 stocks in 5 years
For every round
- Randomly select 20 stocks from sp100
- Backtest using exp3
- Store the exp3 cumulative return and buy and hold return

## Finally conduct pair-wise t-test to see the significance


In [58]:
EXP3_rets = []
BAH_rets = []
num_rounds = 100
for i in range(num_rounds):
    portfolio = np.random.choice(tickers,20)
    temp_data = data[portfolio]
    exp3 = EXP3(temp_data,0.1)
    exp3.backtest(len(temp_data))
    rewards = exp3.data.cumsum()
    EXP3_rets.append(rewards.loc[rewards.index[-1],'exp3'])
    BAH_rets.append(rewards.loc[rewards.index[-1],'bah'])

## With Gamma = 0.1 , 75% of the time, exp3 strategy will out perform buy and hold

In [59]:
(np.array(EXP3_rets)-np.array(BAH_rets) > 0).mean()

0.75

## T test shows significant evidence that exp3 yields higher return than buy and hold

In [60]:
ttest_rel(EXP3_rets,BAH_rets,alternative = 'greater')

Ttest_relResult(statistic=5.0486962826756265, pvalue=1.0137294612295095e-06)